In [2]:
import torch  
import torch.nn as nn
import torch.optim as otim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
Z_DIM = 20
EPOCHS = 20
RESULT_DIR = 'vae_results'


In [5]:
import os
if not os.path.exists(RESULT_DIR):
    os.mkdir(RESULT_DIR)
    

In [6]:
train_dataset = datasets.MNIST(root="./data", train=True, transform=transforms.ToTensor(), download=True)



train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

100%|██████████| 9.91M/9.91M [00:09<00:00, 1.00MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 143kB/s]
100%|██████████| 1.65M/1.65M [00:02<00:00, 675kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 16.8MB/s]


In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim = 784, h_dim=400, z_dim=20):
        super(VAE,self).__init__()


        # 인코더
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, h_dim),
            nn.ReLu()
        )


        # 디코더
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, input_dim),
            nn.Sigmoid()
        )


        # 평균을 위한 레이어
        self.fc_mu = nn.Linear(h_dim, z_dim)
        # 로그 분산을 위한 레이어
        self.fc_log_var = nn.Linear(h_dim, z_dim)


    def forward(self, x):
        h = self.encoder(x)
        mu, log_var = self.fc_mu(h), self.fc_log_var(h)


        # 재파리미터화 기법
        std = torch.exp(0.5 * log_var)
        # 표준정규분포에서 샘플링을 한 값
        eps = torch.randn_like(std)
        z = mu + eps * std


        x_reconstructed = self.decoder(z)
        return x_reconstructed, mu, log_var